<a href="https://colab.research.google.com/github/kms040411/GTSRB_recognition/blob/main/Training_Code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import Libraries

In [41]:
import numpy as np
import matplotlib.pyplot as plt # this is for importing matplotlib.pyplot (library for graph plot)
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import os
import pandas as pd

from PIL import Image, ImageFilter, ImageEnhance, PpmImagePlugin
import random

# Mount Google Drive


In [42]:
from google.colab import files
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Modify Dataset
We apply blurring, gaussian blurring, decolorization, darkening, exposure, adding noise to random image.

## Parameters

In [43]:
#path = "/content/drive/My Drive/CS470/Project/GTSRB_Final_Training_Images/GTSRB/Final_Training/Images/"
label_num = 43
#test_path = "/content/drive/My Drive/CS470/Project/GTSRB_Final_Test_Images/GTSRB/Final_Test/Images/"

## Functions


In [44]:
def do_blur(img):
    blurred = img.filter(ImageFilter.BoxBlur(2))
    return blurred

def do_gaussianblur(img):
    blurred = img.filter(ImageFilter.GaussianBlur(2))
    return blurred

def do_decolorization(img):
    grayscaled = img.convert("LA")
    grayscaled = grayscaled.convert("RGB")
    return grayscaled

def do_darkening(img):
    enhancer = ImageEnhance.Brightness(img)
    darkened = enhancer.enhance(0.3)
    return darkened

def do_exposure(img):
    enhancer = ImageEnhance.Contrast(img)
    contrasted = enhancer.enhance(3.0)
    enhancer2 = ImageEnhance.Brightness(contrasted)
    exposured = enhancer.enhance(3.0)
    return exposured

def do_noise(img):
    noisy = img.effect_spread(2)
    return noisy

# Parameters

In [45]:
iter_num = 100
learning_rate = 0.00005
batch_size = 16

Use_CNN = True
Use_FRCNN = False

is_continue = True
model_path = "/content/drive/My Drive/CS470/Project/pretrained.pth"

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load Dataset

## Local data version

In [46]:
class GSTRB_dataset():
  def __init__(self, root):
    self.root = root  # path for image root folder
    self.basic_transform = data_transforms = transforms.Compose([
                                                                  transforms.Resize((32, 32)),
                                                                  transforms.ToTensor(),
                                                                  transforms.Normalize((0, 0, 0), (1, 1, 1)) # Normalize to mean=0, std=1
                                                                ])

    # Read csv
    self.csv_list = [list() for i in range(label_num)]
    for i in range(0, label_num):
      label_str = None
      if i < 10:
        label_str = "0000" + str(i)
      else:
        label_str = "000" + str(i)
        
      folder_path = self.root + label_str
      with open(folder_path + "/GT-" + label_str + ".csv") as f:
        line = f.readline() # Ignore First line
        while(True):
            line = f.readline()
            if line == "":
                break
            splited = line.split(";")
            file_name = splited[0]
            self.csv_list[i].append(file_name)
    
    # Split train data and validate data
    self.train_list = [None for i in range(label_num)]
    self.validate_list = [None for i in range(label_num)]
    for i in range(0, label_num):
      total_num = len(self.csv_list[i])
      random.shuffle(self.csv_list[i])
      split_index = int(total_num * 0.7)
      self.train_list[i] = self.csv_list[i][:split_index]
      self.validate_list[i] = self.csv_list[i][split_index:]

  # Select random image and its index
  def sample_one(self, data_set):
    # Select label first
    label = random.randrange(0, label_num)
    label_str = None
    if label < 10:
      label_str = "0000" + str(label)
    else:
      label_str = "000" + str(label)

    folder_path = self.root + label_str

    # Sample image
    # It is possible to select the same data, but the probability is very low.
    target = random.sample(data_set[label], 1)[0]
    img = Image.open(folder_path + "/" + target)

    # Apply transform
    transforms = [do_blur, do_gaussianblur, do_decolorization, do_darkening, do_exposure, do_noise, None]
    rand_transform = random.sample(transforms, 1)[0]
    if rand_transform is not None:
      img = rand_transform(img)

    # Basic transfrom
    img = self.basic_transform(img)
    img = torch.unsqueeze(img, 0)
    label = torch.LongTensor([label])

    return (img, label)
  
  def sample_train(self):
    return self.sample_one(self.train_list)
  
  def sample_validate(self):
    return self.sample_one(self.validate_list)

In [47]:
class GSTRB_test_dataset():
  def __init__ (self, root):
    self.root = root  # path for image root folder
    self.basic_transform = data_transforms = transforms.Compose([
                                                                  transforms.Resize((32, 32)),
                                                                  transforms.ToTensor(),
                                                                  transforms.Normalize((0, 0, 0), (1, 1, 1)) # Normalize to mean=0, std=1
                                                                ])
    # Read csv
    self.train_list = list()
    with open(folder_path + "/GT-final_test.GT.csv") as f:
        line = f.readline() # Ignore First line
        while(True):
            line = f.readline()
            if line == "":
                break
            splited = line.split(";")
            file_name = splited[0]
            label = splited[7]
            self.csv_list[i].append((file_name, label))
  
  def sample_test(self):
    target = random.sample(self.train_list, 1)[0]
    img, label = Image.open(self.root + "/" + target)
    label = int(label)

    # Apply transform
    transforms = [do_blur, do_gaussianblur, do_decolorization, do_darkening, do_exposure, do_noise, None]
    #rand_transform = random.sample(transforms, 1)[0]
    #if rand_transform is not None:
    #  img = rand_transform(img)
    
    # Basic transfrom
    img = self.basic_transform(img)
    img = torch.unsqueeze(img, 0)
    label = torch.LongTensor([label])

    return (img, label)

## Kaggle version

In [48]:
# We need to upload kaggle.json

from google.colab import files
files.upload()
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

# Download dataset from kaggle
#!kaggle datasets download -d meowmeowmeowmeowmeow/gtsrb-german-traffic-sign --force
#!unzip -q *.zip

In [49]:
class GSTRB_train_dataset(torch.utils.data.Dataset):
  def __init__(self, root, csv):
    self.root = root
    self.csv = pd.read_csv(csv)
    self.basic_transform = data_transforms = transforms.Compose([
                                                              transforms.Resize((32, 32)),
                                                              transforms.ToTensor(),
                                                              transforms.Normalize((0.3337, 0.3064, 0.3171), ( 0.2672, 0.2564, 0.2629))
                                                            ])
    self.counter = 0
    self.transform_num = 7
    
  def __len__(self):
    ans = len(self.csv) * self.transform_num
    return ans

  def __getitem__(self, idx):
    image = Image.open(os.path.join(self.root, str(self.csv.iloc[idx // self.transform_num, 7])))
    label = torch.tensor(self.csv.iloc[idx // self.transform_num, 6]).long()
        
    # Apply transform
    transforms = [do_blur, do_gaussianblur, do_decolorization, do_darkening, do_exposure, do_noise, None]
    rand_transform = random.sample(transforms, 1)[0]
    rand_transform = transforms[self.counter]
    self.counter = self.counter + 1
    self.counter = self.counter % self.transform_num
    if rand_transform is not None:
      image = rand_transform(image)
    image = self.basic_transform(image)

    pack = (image, label)
    return pack

train_dataset, valid_dataset = None, None
if Use_CNN:
  dataset_loader = GSTRB_train_dataset("/content/", "/content/Train.csv")

  # Split Dataset
  train_Num = int(0.8*len(dataset_loader))
  valid_Num = len(dataset_loader) - train_Num
  train_dataset, valid_dataset = torch.utils.data.random_split(dataset_loader, [train_Num, valid_Num])

  train_dataset = torch.utils.data.DataLoader(train_dataset,
                                          batch_size=batch_size,
                                          shuffle=True,
                                          drop_last=True
                                          )
  valid_dataset = torch.utils.data.DataLoader(valid_dataset,
                                          batch_size=batch_size,
                                          shuffle=False,
                                          drop_last=True
                                          )

## Dataset for Fast RCNN Model

In [50]:
class GTSRB_frcnn_train_dataset(torch.utils.data.Dataset):
  def __init__(self, root, csv):
    self.root = root
    self.csv = pd.read_csv(csv)
    self.basic_transform = data_transforms = transforms.Compose([
                                                              transforms.Resize((32, 32)),
                                                              transforms.ToTensor(),
                                                              transforms.Normalize((0.3337, 0.3064, 0.3171), ( 0.2672, 0.2564, 0.2629))
                                                            ])
    self.counter = 0
    self.transform_num = 7
  
  def __len__(self):
    ans = len(self.csv) * self.transform_num
    return ans
  
  def __getitem__(self, idx):
    image = Image.open(os.path.join(self.root, str(self.csv.iloc[idx // self.transform_num, 7])))
    label = torch.tensor([self.csv.iloc[idx // self.transform_num, 6]]).long()
        
    # Apply transform
    transforms = [do_blur, do_gaussianblur, do_decolorization, do_darkening, do_exposure, do_noise, None]
    rand_transform = random.sample(transforms, 1)[0]
    rand_transform = transforms[self.counter]
    self.counter = self.counter + 1
    self.counter = self.counter % self.transform_num
    if rand_transform is not None:
      image = rand_transform(image)
    image = self.basic_transform(image)

    # Get boxes
    box = torch.tensor([[self.csv.iloc[idx // self.transform_num, 2], self.csv.iloc[idx // self.transform_num, 3], self.csv.iloc[idx // self.transform_num, 4], self.csv.iloc[idx // self.transform_num, 5]]])

    pack = (image, label, box)
    return pack

train_frcnn_dataset, valid_frcnn_dataset = None, None
if Use_FRCNN:
  frcnn_dataset_loader = GTSRB_frcnn_train_dataset("/content/", "/content/Train.csv")

  # Split Dataset
  train_Num = int(0.8*len(frcnn_dataset_loader))
  valid_Num = len(frcnn_dataset_loader) - train_Num
  train_frcnn_dataset, valid_frcnn_dataset = torch.utils.data.random_split(frcnn_dataset_loader, [train_Num, valid_Num])

  train_frcnn_dataset = torch.utils.data.DataLoader(train_frcnn_dataset,
                                          batch_size=batch_size,
                                          shuffle=True,
                                          drop_last=True
                                          )
  valid_frcnn_dataset = torch.utils.data.DataLoader(valid_frcnn_dataset,
                                          batch_size=batch_size,
                                          shuffle=False,
                                          drop_last=True
                                          )

# Define Model

In [64]:
'''class SimpleCNN(nn.Module):
  def __init__(self):
    super(SimpleCNN, self).__init__()
    # Spatial Transformation Network
    self.localization = nn.Sequential(
                                        nn.Conv2d(3, 8, kernel_size=7),
                                        nn.MaxPool2d(2, stride=2),
                                        nn.ReLU(True),
                                        nn.Conv2d(8, 10, kernel_size=5),
                                        nn.MaxPool2d(2, stride=2),
                                        nn.ReLU(True)
                                    )
    
    # Regressor for the affine matrix
    self.fc_loc = nn.Sequential(
                                  nn.Linear(10 * 4 * 4, 32),
                                  nn.ReLU(True),
                                  nn.Linear(32, 3 * 2)
                              )

    # CNN
    self.conv1 = nn.Conv2d(in_channels=3, out_channels=100, kernel_size=5)
    self.relu1 = nn.ReLU()
    self.batchnorm1 = nn.BatchNorm2d(100)
    self.maxpool1 = nn.MaxPool2d(2)
    self.drop1 = nn.Dropout2d()
    self.conv2 = nn.Conv2d(in_channels=100, out_channels=150, kernel_size=3)
    self.relu2 = nn.ReLU()
    self.batchnorm2 = nn.BatchNorm2d(150)
    self.maxpool2 = nn.MaxPool2d(2)
    self.drop2 = nn.Dropout2d()
    self.conv3 = nn.Conv2d(in_channels=150, out_channels=250, kernel_size=3)
    self.relu3 = nn.ReLU()
    self.batchnorm3 = nn.BatchNorm2d(250)
    self.maxpool3 = nn.MaxPool2d(2)
    self.drop3 = nn.Dropout2d()
    

    # Full-Connected
    self.fc1 = nn.Linear(250 * 2 * 2, 350)
    self.relu4 = nn.ReLU()
    self.fc2 = nn.Linear(350, label_num)
    self.relu5 = nn.ReLU()

  def forward(self, x):
    output = x

    # Spatial Transformation Network
    output = self.localization(output)
    output = output.view(-1, 10 * 4 * 4)
    theta = self.fc_loc(output)
    theta = theta.view(-1, 2, 3)
    grid = torch.nn.functional.affine_grid(theta, x.size(), align_corners=False)
    output = torch.nn.functional.grid_sample(x, grid, align_corners=False)

    # CNN
    output = self.conv1(output)
    output = self.relu1(output)
    output = self.maxpool1(output)
    output = self.batchnorm1(output)
    output = self.drop1(output)

    output = self.conv2(output)
    output = self.relu2(output)
    output = self.maxpool2(output)
    output = self.batchnorm2(output)
    output = self.drop2(output)

    output = self.conv3(output)
    output = self.relu3(output)
    output = self.maxpool3(output)
    output = self.batchnorm3(output)
    output = self.drop3(output)

    # Full-Connected
    output = output.view(-1, 250 * 2 * 2)
    output = self.fc1(output)
    output = self.relu4(output)
    output = self.fc2(output)
    output = self.relu5(output)
    output = nn.functional.log_softmax(output, dim=1)
    return output
'''
import torch.nn.functional as F
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        
        # CNN layers
        self.conv1 = nn.Conv2d(3, 100, kernel_size=5)
        self.bn1 = nn.BatchNorm2d(100)
        self.conv2 = nn.Conv2d(100, 150, kernel_size=3)
        self.bn2 = nn.BatchNorm2d(150)
        self.conv3 = nn.Conv2d(150, 250, kernel_size=3)
        self.bn3 = nn.BatchNorm2d(250)
        self.conv_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(250*2*2, 350)
        self.fc2 = nn.Linear(350, label_num)

        self.localization = nn.Sequential(
            nn.Conv2d(3, 8, kernel_size=7),
            nn.MaxPool2d(2, stride=2),
            nn.ReLU(True),
            nn.Conv2d(8, 10, kernel_size=5),
            nn.MaxPool2d(2, stride=2),
            nn.ReLU(True)
            )

        # Regressor for the 3 * 2 affine matrix
        self.fc_loc = nn.Sequential(
            nn.Linear(10 * 4 * 4, 32),
            nn.ReLU(True),
            nn.Linear(32, 3 * 2)
            )
   
        # Initialize the weights/bias with identity transformation
        self.fc_loc[2].weight.data.zero_()
        self.fc_loc[2].bias.data.copy_(torch.tensor([1, 0, 0, 0, 1, 0], dtype=torch.float))


    # Spatial transformer network forward function
    def stn(self, x):
        xs = self.localization(x)
        xs = xs.view(-1, 10 * 4 * 4)
        theta = self.fc_loc(xs)
        theta = theta.view(-1, 2, 3)
        grid = F.affine_grid(theta, x.size())
        x = F.grid_sample(x, grid)
        return x

    def forward(self, x):
        # transform the input
        x = self.stn(x)

        # Perform forward pass
        x = self.bn1(F.max_pool2d(F.leaky_relu(self.conv1(x)),2))
        x = self.conv_drop(x)
        x = self.bn2(F.max_pool2d(F.leaky_relu(self.conv2(x)),2))
        x = self.conv_drop(x)
        x = self.bn3(F.max_pool2d(F.leaky_relu(self.conv3(x)),2))
        x = self.conv_drop(x)
        x = x.view(-1, 250*2*2)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

In [65]:
def weights_init(m):
  if isinstance(m, nn.Linear) or isinstance(m, nn.Conv2d):
    nn.init.normal_(m.weight, mean=0.5, std=1.0)
    nn.init.normal_(m.bias, mean=0.5, std=1.0)

model = None
if Use_CNN:
  model = SimpleCNN()
  model = model.to(device)

In [53]:
import torchvision

frcnn_model = None
if Use_FRCNN:
  frcnn_model = torchvision.models.detection.fasterrcnn_resnet50_fpn(num_classes=label_num)
  frcnn_model = frcnn_model.to(device)

# Training Code

In [54]:
def label(prediction):
  _, pred_label = torch.max(prediction, 1)
  return pred_label

def save_checkpoint(model, optimizer, epoch, model_path):
  state = {
      'state_dict': model,
      'optimizer': optimizer,
      'epoch': epoch
  }

In [ ]:
if Use_CNN:
  optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
  criterion = torch.nn.CrossEntropyLoss()
  start_epoch = 1
  if is_continue:
    print("Load previous model parameters")
    loaded = torch.load(model_path)
    model.load_state_dict(loaded)
    #optimizer.load_state_dict(loaded['optimizer'])
    #start_epoch = loaded['epoch'] + 1
  else:
    model.apply(weights_init)
    # Initalize STN
    model.fc_loc[2].weight.data.zero_()
    model.fc_loc[2].bias.data = torch.tensor([1, 0, 0, 0, 1, 0], dtype=torch.float).cuda()

  # For each epoch, multiply 0.95 to learning rate
  scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma= 0.99)

  # Statistics for total training
  train_loss = np.zeros(iter_num, dtype=float)
  validation_loss = np.zeros(iter_num, dtype=float)
  train_accuracy = np.zeros(iter_num, dtype=float)
  validation_accuracy = np.zeros(iter_num, dtype=float)

  previous_best_accurary = None

  for iter in range(start_epoch, iter_num + start_epoch):
    # Set training mode
    model.train()

    # Statistics for current iteration
    current_loss = 0.0
    total_prediction = 0.0
    correct_prediction = 0.0
    i = 0
    for _, (x, answer) in enumerate(train_dataset):
      print("\r", end="")
      i = i + 1
      print("train_dataset: ",i, "/", len(train_dataset), end="")
      x, answer = x.cuda(), answer.cuda()
      optimizer.zero_grad()
      prediction = model(x)
      loss = criterion(prediction, answer)  # We use cross entropy loss for loss calculation
      loss.backward()
      optimizer.step()

      current_loss += loss.item()
      total_prediction += x.data.size(0)
      correct_prediction += (label(prediction) == answer.data).sum().item()
    
    accuracy = correct_prediction * 100.0 / total_prediction
    train_loss[iter - start_epoch] = current_loss / total_prediction
    train_accuracy[iter - start_epoch] = accuracy

    scheduler.step() 

    if iter % 1 == 0:
      print(str(iter) + "/" + str(iter_num + start_epoch) + ": train loss - " + str(train_loss[iter - start_epoch]) + ", train accuracy - " + str(train_accuracy[iter - start_epoch]) + "%, lr: " + str(scheduler.get_last_lr()))

    # Set validation mode
    model.eval()

    # Statistics for current iteration
    current_loss = 0.0
    total_prediction = 0.0
    correct_prediction = 0.0
    #for batch_num in range(batch_size):
    #  x, answer = dataset.sample_validate()
    with torch.no_grad():
      i=0
      for _, (x, answer) in enumerate(valid_dataset):
        print("\r", end="")
        i = i + 1
        print("valid_dataset: ",i, "/", len(valid_dataset), end="")
        x, answer = x.cuda(), answer.cuda()
        prediction = model(x)
        loss = criterion(prediction, answer)  # We use cross entropy loss for loss calculation

        current_loss += loss.item()
        total_prediction += x.data.size(0)
        correct_prediction += (label(prediction) ==  answer.data).sum().item()
      
      accuracy = correct_prediction * 100.0 / total_prediction
      train_loss[iter - start_epoch] = current_loss / total_prediction
      train_accuracy[iter - start_epoch] = accuracy

    if iter % 1 == 0:
      print(str(iter) + "/" + str(iter_num + start_epoch) + ": validation loss - " + str(train_loss[iter - start_epoch]) + ", validation accuracy - " + str(train_accuracy[iter - start_epoch]) + "%")
    
    # Save checkpoint
    if iter % 50 == 0:
      print("Save Current Model Parameter")
      save_checkpoint(model.state_dict(), optimizer.state_dict(), iter, model_path)

Load previous model parameters
train_dataset:  7 / 13723

/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:3448: UserWarning: Default grid_sample and affine_grid behavior has changed to align_corners=False since 1.3.0. Please specify align_corners=True if the old behavior is desired. See the documentation of grid_sample for details.
  warnings.warn("Default grid_sample and affine_grid behavior has changed "
/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:3385: UserWarning: Default grid_sample and affine_grid behavior has changed to align_corners=False since 1.3.0. Please specify align_corners=True if the old behavior is desired. See the documentation of grid_sample for details.
  warnings.warn("Default grid_sample and affine_grid behavior has changed "


train_dataset:  6280 / 13723

## Train Fast RCNN

In [ ]:
if Use_FRCNN:
  optimizer = torch.optim.Adam(frcnn_model.parameters(), lr=learning_rate)
  start_epoch = 1
  if is_continue:
    print("Load previous model parameters")
    loaded = torch.load(model_path)
    model.load_state_dict(loaded['state_dict'])
    optimizer.load_state_dict(loaded['optimizer'])
    start_epoch = loaded['epoch'] + 1

  # For each epoch, multiply 0.95 to learning rate
  scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma= 0.99)

  # Statistics for total training
  train_loss = np.zeros(iter_num, dtype=float)
  validation_loss = np.zeros(iter_num, dtype=float)
  train_accuracy = np.zeros(iter_num, dtype=float)
  validation_accuracy = np.zeros(iter_num, dtype=float)

  previous_best_accurary = None

  for iter in range(start_epoch, iter_num + start_epoch):
    # Set training mode
    frcnn_model.train()

    # Statistics for current iteration
    current_loss = 0.0
    total_prediction = 0.0
    correct_prediction = 0.0
    i = 0
    for _, (x, label, box) in enumerate(train_frcnn_dataset):
      print("\r", end="")
      i = i + 1
      print(i, "/", len(train_frcnn_dataset), end="")
      x, label, box = x.to(device), label.to(device), box.to(device)
      images = list(image for image in x)
      targets = []
      for i in range(len(images)):
        d = {}
        d['boxes'] = box[i]
        d['labels'] = label[i]
        targets.append(d)

      prediction = frcnn_model(images, targets)
      optimizer.zero_grad()
      loss = prediction['loss_classifier'].mean() + prediction['loss_classifier'].mean() + prediction['loss_objectness'].mean() + prediction['loss_rpn_box_reg'].mean()
      loss.backward()
      optimizer.step()

      current_loss += loss.item()
      total_prediction += x.data.size(0)
      #correct_prediction += (label( == answer.data).sum().item()
    
    accuracy = correct_prediction * 100.0 / total_prediction
    train_loss[iter - start_epoch] = current_loss / total_prediction
    train_accuracy[iter - start_epoch] = accuracy

    scheduler.step() 
    
    if iter % 1 == 0:
      print(str(iter) + "/" + str(iter_num + start_epoch) + ": train loss - " + str(train_loss[iter - start_epoch]) + ", train accuracy - " + str(train_accuracy[iter - start_epoch]) + "%, lr: " + str(scheduler.get_last_lr()))

    # Set validation mode
    frcnn_model.eval()

    # Statistics for current iteration
    current_loss = 0.0
    total_prediction = 0.0
    correct_prediction = 0.0
    #for batch_num in range(batch_size):
    #  x, answer = dataset.sample_validate()
    with torch.no_grad():
      for _, (x, label, box) in enumerate(valid_frcnn_dataset):
        x, label, box = x.to(device), label.to(device), box.to(device)
        images = list(image for image in x)
        targets = []
        for i in range(len(images)):
          d = {}
          d['boxes'] = box[i]
          d['labels'] = label[i]
          targets.append(d)

        prediction = frcnn_model(images, targets)
        loss = prediction['loss_classifier'].mean() + prediction['loss_classifier'].mean() + prediction['loss_objectness'].mean() + prediction['loss_rpn_box_reg'].mean()

        current_loss += loss.item()
        total_prediction += x.data.size(0)
        #correct_prediction += (label(prediction) ==  answer.data).sum().item()
      
      accuracy = correct_prediction * 100.0 / total_prediction
      train_loss[iter - start_epoch] = current_loss / total_prediction
      train_accuracy[iter - start_epoch] = accuracy

    if iter % 1 == 0:
      print(str(iter) + "/" + str(iter_num + start_epoch) + ": validation loss - " + str(train_loss[iter - start_epoch]) + ", validation accuracy - " + str(train_accuracy[iter - start_epoch]) + "%")
    
    # Save checkpoint
    if iter % 50 == 0:
      print("Save Current Model Parameter")
      save_checkpoint(frcnn_model.state_dict(), optimizer.state_dict(), iter, model_path)
      